# カメラの映像を取得してみよう

ここでは、AKARIのヘッドのOAK-D Liteのカメラ映像の取得方法を説明します。

## 1. RGB画像の取得
まずはOAK-D LiteのRGB画像の取得を試してみましょう。  
OAK-Dは使い方がやや特殊です。まず、下記のセルを実行してRGB画像を取得するためのOAK-Dのセットアップをします。


In [ ]:
import cv2
import depthai as dai
%matplotlib inline
import matplotlib.pyplot as plt

# OAK-Dのパイプライン作成
pipeline = dai.Pipeline()

# ソースとアウトプットの設定
cam_rgb = pipeline.create(dai.node.ColorCamera)
xout_video = pipeline.create(dai.node.XLinkOut)

# RGBのカメラ、1080P、解像度1920x1080を指定
cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
cam_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam_rgb.setVideoSize(1920, 1080)
# ソースとアウトプットを接続
cam_rgb.video.link(xout_video.input)

# キューのブロッキングなし、キューサイズ１を指定
xout_video.input.setBlocking(False)
xout_video.input.setQueueSize(1)
# ストリーミング名設定
xout_video.setStreamName("video")

cam_rgb.preview.link(xout_video.input)

次に下記のセルを実行することで、画像を取得して表示することができます。

In [ ]:
# デバイスをパイプラインに接続
with dai.Device(pipeline) as device:
    # フレームを取得して表示
    video = device.getOutputQueue(name="video", maxSize=1, blocking=False)  # type: ignore
    frame = video.get().getCvFrame()
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.show()

## 2. ステレオdepth画像の取得
次にOAK-D LiteのステレオRGBを用いたdepth画像の取得を試してみましょう。  
まず、下記のセルを実行してステレオ画像を取得するためのOAK-Dのセットアップをします。

In [ ]:
import cv2
import numpy
import depthai as dai
%matplotlib inline
import matplotlib.pyplot as plt

MAX_DEPTH = 1500.0  # unit: [mm]

# OAK-Dのパイプライン作成
pipeline = dai.Pipeline()

# ソースとアウトプットの設定
cam_left = pipeline.create(dai.node.MonoCamera)
cam_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)
cam_right = pipeline.create(dai.node.MonoCamera)
cam_right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)
stereo = pipeline.create(dai.node.StereoDepth)
xout_depth = pipeline.create(dai.node.XLinkOut)

# ストリーミング名設定
xout_depth.setStreamName("depth")

# ソースとアウトプットを接続
cam_left.out.link(stereo.left)
cam_right.out.link(stereo.right)
stereo.depth.link(xout_depth.input)


次に下記のセルを実行することで、画像を取得して表示することができます。

In [ ]:
# デバイスをパイプラインに接続
with dai.Device(pipeline) as device:
    video = device.getOutputQueue(name="depth", maxSize=4, blocking=False)  # type: ignore
    depth = video.get().getCvFrame()
    # フレームをカラーマップに割り当て
    depth[numpy.where(depth > MAX_DEPTH)] = 0
    norm_depth = (depth * (255 / MAX_DEPTH)).astype(numpy.uint8)
    colored_depth: numpy.ndarray = cv2.applyColorMap(norm_depth, cv2.COLORMAP_JET)
    plt.imshow(colored_depth)
    plt.show()

## 最後に
以上でカメラ画像取得のチュートリアルは終わりです。  
OAK-Dの使い方はここでは説明していない内容が多数あります。詳しくは[depthaiのマニュアル](https://docs.luxonis.com/en/latest/)をご覧ください。  
AKARIのチュートリアルは以上となります。